<a href="https://colab.research.google.com/github/Natotela/Bemba/blob/master/Thin_Plate_Spline_Motion_Model_for_SD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title # Step 1: Setup
!git clone https://github.com/yoyo-nb/Thin-Plate-Spline-Motion-Model.git

%cd Thin-Plate-Spline-Motion-Model
!mkdir checkpoints

!wget -c https://cloud.tsinghua.edu.cn/f/da8d61d012014b12a9e4/?dl=1 -O checkpoints/vox.pth.tar
#!wget -c https://cloud.tsinghua.edu.cn/f/483ef53650b14ac7ae70/?dl=1 -O checkpoints/ted.pth.tar
#!wget -c https://cloud.tsinghua.edu.cn/f/9ec01fa4aaef423c8c02/?dl=1 -O checkpoints/taichi.pth.tar
#!wget -c https://cloud.tsinghua.edu.cn/f/cd411b334a2e49cdb1e2/?dl=1 -O checkpoints/mgif.pth.tar

Cloning into 'Thin-Plate-Spline-Motion-Model'...
remote: Enumerating objects: 112, done.
remote: Counting objects: 100% (20/20), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 112 (delta 10), reused 13 (delta 8), pack-reused 92
Receiving objects: 100% (112/112), 32.66 MiB | 21.73 MiB/s, done.
Resolving deltas: 100% (44/44), done.
/content/Thin-Plate-Spline-Motion-Model
--2023-01-06 16:21:02--  https://cloud.tsinghua.edu.cn/f/da8d61d012014b12a9e4/?dl=1
Resolving cloud.tsinghua.edu.cn (cloud.tsinghua.edu.cn)... 166.111.6.101, 2402:f000:1:406:166:111:6:101
Connecting to cloud.tsinghua.edu.cn (cloud.tsinghua.edu.cn)|166.111.6.101|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cloud.tsinghua.edu.cn/seafhttp/files/543531ae-3cd4-468a-86fb-1375b79b35d2/vox.pth.tar [following]
--2023-01-06 16:21:07--  https://cloud.tsinghua.edu.cn/seafhttp/files/543531ae-3cd4-468a-86fb-1375b79b35d2/vox.pth.tar
Reusing existing connection to cloud.tsi

In [2]:
#@title # Step 2: Settings
#@markdown ## Import your driving video and/or image before filling in the form
#@markdown #### # Note: paths are relative to the 'Thin-Plate-Spline-Motion-Model' folder

import torch

# edit the config
device = torch.device('cuda:0')
dataset_name = 'vox' # ['vox', 'taichi', 'ted', 'mgif']
source_image_path = './assets/source.png' #@param {type:"string"}
driving_video_path = './assets/driving.mp4' #@param {type:"string"}
config_path = 'config/vox-256.yaml'
checkpoint_path = 'checkpoints/vox.pth.tar'
predict_mode = 'relative' # ['standard', 'relative', 'avd']
find_best_frame = True # when use the relative mode to animate a face, use 'find_best_frame=True' can get better quality result

pixel = 256 # for vox, taichi and mgif, the resolution is 256*256
if(dataset_name == 'ted'): # for ted, the resolution is 384*384
    pixel = 384

if find_best_frame:
  !pip install face_alignment

output_video_path = './generated.mp4'
try:
  import imageio
  import imageio_ffmpeg
except:
  !pip install imageio_ffmpeg
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from skimage.transform import resize
from IPython.display import HTML
import warnings
import os

warnings.filterwarnings("ignore")

source_image = imageio.imread(source_image_path)
reader = imageio.get_reader(driving_video_path)

source_image = resize(source_image, (pixel, pixel))[..., :3]

fps = reader.get_meta_data()['fps']
driving_video = []
try:
    for im in reader:
        driving_video.append(im)
except RuntimeError:
    pass
reader.close()

driving_video = [resize(frame, (pixel, pixel))[..., :3] for frame in driving_video]

def display(source, driving, generated=None):
    fig = plt.figure(figsize=(8 + 4 * (generated is not None), 6))

    ims = []
    for i in range(len(driving)):
        cols = [source]
        cols.append(driving[i])
        if generated is not None:
            cols.append(generated[i])
        im = plt.imshow(np.concatenate(cols, axis=1), animated=True)
        plt.axis('off')
        ims.append([im])

    ani = animation.ArtistAnimation(fig, ims, interval=50, repeat_delay=1000)
    plt.close()
    return ani
    

HTML(display(source_image, driving_video).to_html5_video())

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for face_alignment: filename=face_alignment-1.3.5-py2.py3-none-any.whl size=28242 sha256=69d48b497340147896989efe6403fef54ad48d832d2ae8c69c97164ccf4f29b3
  Stored in directory: /root/.cache/pip/wheels/32/21/d8/7bf61c86e3a6c5af8f118708d4c9c26512dbd1c913a0b97c8a
Successfully built face_alignment
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.9/26.9 MB 56.9 MB/s eta 0:00:00


In [ ]:
#@title # Step 3: Run Thin-Plate-Spline-Motion-Model
from demo import load_checkpoints
inpainting, kp_detector, dense_motion_network, avd_network = load_checkpoints(config_path = config_path, checkpoint_path = checkpoint_path, device = device)

from demo import make_animation
from skimage import img_as_ubyte

if predict_mode=='relative' and find_best_frame:
    from demo import find_best_frame as _find
    i = _find(source_image, driving_video, device.type=='cpu')
    print ("Best frame: " + str(i))
    driving_forward = driving_video[i:]
    driving_backward = driving_video[:(i+1)][::-1]
    predictions_forward = make_animation(source_image, driving_forward, inpainting, kp_detector, dense_motion_network, avd_network, device = device, mode = predict_mode)
    predictions_backward = make_animation(source_image, driving_backward, inpainting, kp_detector, dense_motion_network, avd_network, device = device, mode = predict_mode)
    predictions = predictions_backward[::-1] + predictions_forward[1:]
else:
    predictions = make_animation(source_image, driving_video, inpainting, kp_detector, dense_motion_network, avd_network, device = device, mode = predict_mode)

#save resulting video
imageio.mimsave(output_video_path, [img_as_ubyte(frame) for frame in predictions], fps=fps)

HTML(display(source_image, driving_video, predictions).to_html5_video())

Downloading: "https://www.adrianbulat.com/downloads/python-fan/s3fd-619a316812.pth" to /root/.cache/torch/hub/checkpoints/s3fd-619a316812.pth


  0%|          | 0.00/85.7M [00:00<?, ?B/s]

Downloading: "https://www.adrianbulat.com/downloads/python-fan/2DFAN4-cd938726ad.zip" to /root/.cache/torch/hub/checkpoints/2DFAN4-cd938726ad.zip


  0%|          | 0.00/91.9M [00:00<?, ?B/s]

78it [00:04, 15.99it/s]


Best frame: 51


100%|██████████| 52/52 [00:03<00:00, 14.88it/s]


In [ ]:
#@title # Step 4: upsize and export video
#@markdown ## Output file is upsized.mp4
%cd ..
!apt install ffmpeg
!apt install libmagic1 python3-yaml
!apt install libvulkan-dev
!pip install --user youtube-dl
!wget https://github.com/k4yt3x/video2x/archive/refs/tags/4.7.0.tar.gz

!tar -xvf 4.7.0.tar.gz
%cd video2x-4.7.0/src
!pip install -r /content/video2x-4.7.0/src/requirements.txt
!rm -rf /content/video2x-4.7.0/src/video2x.yaml
!wget -O /content/video2x-4.7.0/src/video2x.yaml https://raw.githubusercontent.com/lenardcarroll/video2x.yaml/main/video2x.yaml
%cd ../..
!wget https://github.com/nihui/realsr-ncnn-vulkan/releases/download/20200818/realsr-ncnn-vulkan-20200818-linux.zip
!unzip realsr-ncnn-vulkan-20200818-linux.zip
# !wget https://github.com/nihui/srmd-ncnn-vulkan/releases/download/20200818/srmd-ncnn-vulkan-20200818-linux.zip
# !unzip srmd-ncnn-vulkan-20200818-linux
# !wget https://github.com/nihui/waifu2x-ncnn-vulkan/releases/download/20200818/waifu2x-ncnn-vulkan-20200818-linux.zip
# !unzip waifu2x-ncnn-vulkan-20200818-linux.zip
!rm *-linux.zip
!pip install -U PyYAML

!python video2x-4.7.0/src/video2x.py -i ./Thin-Plate-Spline-Motion-Model/generated.mp4 -o ./upsized.mp4 -d realsr_ncnn_vulkan -h 512 -w 512

/content
Reading package lists... Done
Building dependency tree       
Reading state information... Done
ffmpeg is already the newest version (7:3.4.11-0ubuntu0.1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 5 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  libmagic-mgc
Suggested packages:
  file
The following NEW packages will be installed:
  libmagic-mgc libmagic1 python3-yaml
0 upgraded, 3 newly installed, 0 to remove and 5 not upgraded.
Need to get 362 kB of archives.
After this operation, 5,684 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/main amd

In [ ]:
#@title # Step 5 (optional): Save frames
!rm -rf frames
!rm frames.zip
!mkdir frames
!ffmpeg -i upsized.mp4 frames/out%03d.png
!zip -r frames.zip frames/

rm: cannot remove 'frames.zip': No such file or directory
ffmpeg version 3.4.11-0ubuntu0.1 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --en

In [ ]:
#@title # Step 6 (optional): fix face with GFPGAN
!rm -rf fixed
!mkdir fixed

#installing the dependencies
# Install pytorch
!pip install torch torchvision

# Check torch and cuda versions
import torch
print('Torch Version: ', torch.__version__)
print('CUDA Version: ', torch.version.cuda)
print('CUDNN Version: ', torch.backends.cudnn.version())
print('CUDA Available:', torch.cuda.is_available())


# Install basicsr - https://github.com/xinntao/BasicSR
# We use BasicSR for both training and inference. 
# Set BASICSR_EXT=True to compile the cuda extensions in the BasicSR - It may take several minutes to compile, please be patient.
!BASICSR_EXT=True pip install basicsr


# Install facexlib - https://github.com/xinntao/facexlib
# We use face detection and face restoration helper in the facexlib package
!pip install facexlib
!mkdir -p /usr/local/lib/python3.7/dist-packages/facexlib/weights  # for pre-trained models


!rm -rf GFPGAN
!git clone https://github.com/TencentARC/GFPGAN.git
%cd GFPGAN

# install extra requirements
!pip install -r requirements.txt

!python setup.py develop

# If you want to enhance the background (non-face) regions with Real-ESRGAN,
# you also need to install the realesrgan package
!pip install realesrgan


#loading the pretrained GAN Models
!wget https://github.com/TencentARC/GFPGAN/releases/download/v0.1.0/GFPGANv1.pth -P experiments/pretrained_models


!python inference_gfpgan.py -i ../frames -o ../fixed --aligned

%cd ..

!rm fixed.zip
!zip -r fixed.zip fixed/restored_faces

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Torch Version:  1.12.1+cu113
CUDA Version:  11.3
CUDNN Version:  8302
CUDA Available: True
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 172 kB 9.4 MB/s 
     |████████████████████████████████| 6.0 MB 48.8 MB/s 
     |████████████████████████████████| 190 kB 51.2 MB/s 
  Created wheel for basicsr: filename=basicsr-1.4.2-cp37-cp37m-linux_x86_64.whl size=4432517 sha256=55f33bdeb5e22445d7f0f6793447e771dce2e89aa36b3bf7fc7af1776de88b72
  Stored in directory: /root/.cache/pip/wheels/2c/b3/4a/e2bc545f98417b6766ca50dd82b2a1f2b37780c68d41da9ca9
Successfully built basicsr
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 59 kB 1.4 MB/s 
     |████████████████████████████████| 177 kB 57.5 MB/s 
  Created wheel for filter

In [ ]:
#@title # Step 7 (optional): recombine to video
!ffmpeg -framerate 20 -pattern_type glob -i 'fixed/restored_faces/*.png' \
  -c:v libx264 -pix_fmt yuv420p out.mp4

ffmpeg version 3.4.11-0ubuntu0.1 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-li